In [1]:
#Various functions to try and pull out meaningful conclusions from the data
# at the moment:
# 1. Can work out best combinations of cards for portfolio using current score and previous tournament scores
# 2. Can calculate scores of current entries based on current score and previous tournamnet scores

# Should look to 
# 1. Calculate best combination of cards for various tournament entries using set limitations like cost

In [9]:
import os

print("Current Working Directory:", os.getcwd())
os.chdir(r'C:\Users\beuzi\OneDrive\Jupyter Notebooks\FantasyTop')


Current Working Directory: C:\Users\beuzi


## Imports and Setup

In [7]:
import os
import re
import pandas as pd
from datetime import datetime
from tqdm import tqdm
from itertools import combinations


# Constants
DATA_FOLDER = r'C:\Users\beuzi\OneDrive\Jupyter Notebooks\FantasyTop\data'

TOURNAMENT_COLUMNS = [
    'Main 12', 'Main 11', 'Main 10', 'Main 9', 'Main 8', 'Main 7', 'Main 5', 'Main 4'
]
ALL_SCORES = [
    'hero_fantasy_score', 'Main 12','Main 11', 'Main 10', 'Main 9', 'Main 8', 'Main 7', 'Main 6 *Sat/Sun Only*', 
    'Main 5', 'All Rarities | 22 days', 'Main 4', 'Main 3', 'Common Only ✳️ Capped 20 🌟', 
    'Rare Only 💠', 'Main 2', 'Main 1', 'Flash Tournament', 'Average', 'Main_Tournaments_Ave', 'Main_Last_4_Ave'
]

# Functions

def get_latest_csv_files(folder_path):
    csv_files = {}
    pattern = re.compile(r'^(.*)_(\d{6}_\d{4})\.csv$')
    
    for file_name in os.listdir(folder_path):
        if file_name.endswith('.csv'):
            match = pattern.match(file_name)
            if match:
                prefix = match.group(1)
                timestamp_str = match.group(2)
                try:
                    timestamp = datetime.strptime(timestamp_str, '%y%m%d_%H%M')
                    if prefix not in csv_files or timestamp > csv_files[prefix][1]:
                        csv_files[prefix] = (file_name, timestamp)
                except Exception as e:
                    print(f"Error processing file {file_name}: {e}")
    return {prefix: os.path.join(folder_path, file_name) for prefix, (file_name, _) in csv_files.items()}

def import_latest_csv_files(folder_path):
    latest_files = get_latest_csv_files(folder_path)
    return {prefix: pd.read_csv(file_path) for prefix, file_path in latest_files.items()}

def calculate_tournament_averages(df):
    if 'Name' in df.columns and 'Handle' in df.columns:
        numeric_df = df.drop(['Name', 'Handle'], axis=1).apply(pd.to_numeric, errors='coerce')
        df['Average'] = numeric_df.mean(axis=1)
        df['Main_Tournaments_Ave'] = numeric_df[TOURNAMENT_COLUMNS].mean(axis=1)
        df['Main_Last_4_Ave'] = numeric_df[TOURNAMENT_COLUMNS[:4]].mean(axis=1)
    else:
        raise KeyError("Columns 'Name' and 'Handle' not found in the DataFrame")
    return df

def reorder_basic_hero_stats(df):
    columns_order = ['current_rank', 'hero_name', 'hero_handle'] + \
        [col for col in df.columns if col not in ['current_rank', 'hero_name', 'hero_handle']]
    return df[columns_order]

def merge_dataframes(dataframes):
    # Checking 'tournament_scores' DataFrame
    print("Checking 'tournament_scores' DataFrame:")
    if 'Handle' in dataframes['tournament_scores'].columns:
        print(" - 'Handle' found in 'tournament_scores'.")
    else:
        print(" - ERROR: 'Handle' NOT found in 'tournament_scores'.")
        print("Columns:", dataframes['tournament_scores'].columns)
        print(dataframes['tournament_scores'].head())
        return

    # Merging dataframes
    merged_supply_listings = dataframes['hero_card_supply'].merge(dataframes['listings'], on='hero_id', how='left')
    merged_with_basic_hero_stats = dataframes['basic_hero_stats'].merge(merged_supply_listings, on='hero_handle', how='left')
    merged_df = merged_with_basic_hero_stats.merge(dataframes['last_trades'], on='hero_id', how='left')
    
    # Check before final merge
    print("Checking columns before final merge with 'tournament_scores':")
    print("Columns in 'merged_df':", merged_df.columns)
    print("Columns in 'tournament_scores':", dataframes['tournament_scores'].columns)
    
    # Merge with 'tournament_scores'
    final_merged_df = merged_df.merge(dataframes['tournament_scores'], left_on='hero_handle', right_on='Handle', how='left')

    # Ensure 'hero_handle' is retained
    if 'hero_handle' not in final_merged_df.columns:
        print(" - ERROR: 'hero_handle' was not retained in the final merge. Renaming columns.")
        final_merged_df['hero_handle'] = final_merged_df['Handle']  # Manually set it back if missing
        print("Final columns after renaming:", final_merged_df.columns)

    # Final merge with hero_stats
    final_merged_df = final_merged_df.merge(dataframes['hero_stats'][['hero_handle', 'inflation_degree']], 
                                            left_on='hero_handle', right_on='hero_handle', how='left')

    print("Final merge complete.")
    final_merged_df['hero_id'] = final_merged_df['hero_id'].astype(str)
    return final_merged_df




def save_final_dataframes(final_merged_df, portfolio_scores):
    final_merged_df.to_csv(f'{DATA_FOLDER}/allHeroData.csv', index=False)
    portfolio_scores.to_csv(f'{DATA_FOLDER}/portfolio.csv', index=False)

def process_portfolio_scores(portfolio_df, final_merged_df):
    merged_df = portfolio_df.merge(final_merged_df, on='hero_handle', how='left')
    portfolio_scores = merged_df.drop(['hero_name_y', 'hero_stars_y', 'hero_followers_count_y', 'hero_profile_image_url_y', 'token_id'], axis=1)
    portfolio_scores.columns = [col.replace('_x', '') for col in portfolio_scores.columns]
    
    columns_order = ['hero_name', 'hero_handle'] + [col for col in portfolio_scores.columns if col not in ['hero_name', 'hero_handle']]
    portfolio_scores = portfolio_scores[columns_order]
    
    portfolio_scores['lastSalePrice'] = portfolio_scores.apply(lambda row: row[f'rarity{row["rarity"]}lastSalePrice'], axis=1)
    portfolio_scores['lowestPrice'] = portfolio_scores.apply(lambda row: row[f'rarity{row["rarity"]}_lowest_price'], axis=1)
    portfolio_scores['rarityCount'] = portfolio_scores.apply(lambda row: row[f'rarity{row["rarity"]}Count'], axis=1)
    
    columns_to_drop = [
        'rarity1_lowest_price', 'rarity2_lowest_price', 'rarity3_lowest_price', 'rarity4_lowest_price',
        'rarity1lastSalePrice', 'rarity2lastSalePrice', 'rarity3lastSalePrice', 'rarity4lastSalePrice',
        'rarity1Count', 'rarity2Count', 'rarity3Count', 'rarity4Count',
        'rarity1_order_count', 'rarity2_order_count', 'rarity3_order_count', 'rarity4_order_count',
        'rarity1lastSaleTime', 'rarity2lastSaleTime', 'rarity3lastSaleTime', 'rarity4lastSaleTime'
    ]
    portfolio_scores = portfolio_scores.drop(columns=columns_to_drop)
    
    for col in ALL_SCORES:
        portfolio_scores[col] = pd.to_numeric(portfolio_scores[col], errors='coerce')
        portfolio_scores[col] = portfolio_scores.apply(lambda row: row[col] * 1.5 if row['hero_rarity_index'].endswith('3') else row[col], axis=1)
    
    return portfolio_scores

# Main Execution

def main():
    dataframes = import_latest_csv_files(DATA_FOLDER)
    dataframes['tournament_scores'] = calculate_tournament_averages(dataframes['tournament_scores'])
    dataframes['basic_hero_stats'] = reorder_basic_hero_stats(dataframes['basic_hero_stats'])
    final_merged_df = merge_dataframes(dataframes)
    portfolio_scores = process_portfolio_scores(dataframes['portfolio'], final_merged_df)
    save_final_dataframes(final_merged_df, portfolio_scores)

if __name__ == "__main__":
    main()


Checking 'tournament_scores' DataFrame:
 - 'Handle' found in 'tournament_scores'.
Checking columns before final merge with 'tournament_scores':
Columns in 'merged_df': Index(['current_rank', 'hero_name', 'hero_handle', 'views', 'tweet_count',
       'fantasy_score', 'reach', 'avg_views', 'hero_followers_count',
       'hero_profile_image_url', 'hero_volume', 'hero_id', 'rarity1Count',
       'rarity2Count', 'rarity3Count', 'rarity4Count', 'burnedCardsCount',
       'utilityCount', 'hero_stars', 'rarity1_lowest_price',
       'rarity2_lowest_price', 'rarity3_lowest_price', 'rarity4_lowest_price',
       'rarity1_order_count', 'rarity2_order_count', 'rarity3_order_count',
       'rarity4_order_count', 'rarity1lastSalePrice', 'rarity2lastSalePrice',
       'rarity3lastSalePrice', 'rarity4lastSalePrice', 'rarity1lastSaleTime',
       'rarity2lastSaleTime', 'rarity3lastSaleTime', 'rarity4lastSaleTime'],
      dtype='object')
Columns in 'tournament_scores': Index(['Unnamed: 0', 'Name', 'Hand

In [14]:
dataframes = import_latest_csv_files(DATA_FOLDER)
dataframes['basic_hero_stats']

,current_rank,hero_name,hero_handle,views,tweet_count,fantasy_score,reach,avg_views,hero_followers_count,hero_profile_image_url,hero_volume
0,1,greg,greg16676935420,81523319,117,1000.000000,286.803798,696780.506403,1017227,https://pbs.twimg.com/profile_images/158101430...,212.246551
1,2,GBH 🦁☝️ Remilio | ᚱᚹᛟ,xbtGBH,23941798,74,959.813585,467.486147,323537.812826,142979,https://pbs.twimg.com/profile_images/177972188...,174.187213
2,3,wallstreetbets,wallstreetbets,38745662,194,955.465391,101.985589,199719.909273,810707,https://pbs.twimg.com/profile_images/181575487...,16.161170
3,4,mitch (rtrd/acc),idrawline,5602370,125,939.614744,105.854892,44818.961453,65075,https://pbs.twimg.com/profile_images/177494222...,0.000000
4,5,based16z,based16z,3835380,267,916.626785,57.048130,14364.719070,35984,https://pbs.twimg.com/profile_images/179782076...,77.616767
...,...,...,...,...,...,...,...,...,...,...,...
182,183,Pentoshi 🐧 euroPeng 🇪🇺,Pentosh1,0,0,0.000000,0.000000,0.000000,800562,https://pbs.twimg.com/profile_images/176736682...,29.874844
183,183,Tom Schmidt ＞|＜,tomhschmidt,0,0,0.000000,0.000000,0.000000,34321,https://pbs.twimg.com/profile_images/177372148...,273.743174
184,183,natealex,natealexnft,0,0,0.000000,0.000000,0.000000,178429,https://pbs.twimg.com/profile_images/174301093...,167.663217
185,183,Kofi,0xKofi,0,0,0.000000,0.000000,0.000000,32354,https://pbs.twimg.com/profile_images/138786910...,55.854240


In [16]:
merged_supply_listings = dataframes['hero_card_supply'].merge(dataframes['listings'], on='hero_id', how='left')
columns_to_drop = ['hero_name', 'hero_followers_count', 'current_rank', 'previous_rank', 'views', 'fantasy_score']
merged_supply_listings = merged_supply_listings.drop(columns=[col for col in columns_to_drop if col in merged_supply_listings.columns])

In [18]:
merged_with_basic_hero_stats = dataframes['basic_hero_stats'].merge(merged_supply_listings, on='hero_handle', how='left')

In [19]:
 merged_with_basic_hero_stats 

,current_rank,hero_name,hero_handle,views,tweet_count,fantasy_score,reach,avg_views,hero_followers_count,hero_profile_image_url,...,utilityCount,hero_stars,rarity1_lowest_price,rarity2_lowest_price,rarity3_lowest_price,rarity4_lowest_price,rarity1_order_count,rarity2_order_count,rarity3_order_count,rarity4_order_count
0,1,greg,greg16676935420,81523319,117,1000.000000,286.803798,696780.506403,1017227,https://pbs.twimg.com/profile_images/158101430...,...,8.0,7.0,NaN,12.0000,1.6500,0.380000,NaN,1.0,1.0,4.0
1,2,GBH 🦁☝️ Remilio | ᚱᚹᛟ,xbtGBH,23941798,74,959.813585,467.486147,323537.812826,142979,https://pbs.twimg.com/profile_images/177972188...,...,8.0,7.0,NaN,NaN,NaN,0.492970,NaN,0.0,0.0,6.0
2,3,wallstreetbets,wallstreetbets,38745662,194,955.465391,101.985589,199719.909273,810707,https://pbs.twimg.com/profile_images/181575487...,...,14.0,7.0,NaN,NaN,1.5000,0.230000,NaN,NaN,2.0,5.0
3,4,mitch (rtrd/acc),idrawline,5602370,125,939.614744,105.854892,44818.961453,65075,https://pbs.twimg.com/profile_images/177494222...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,based16z,based16z,3835380,267,916.626785,57.048130,14364.719070,35984,https://pbs.twimg.com/profile_images/179782076...,...,4.0,7.0,NaN,NaN,0.9000,0.169000,NaN,NaN,2.0,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
182,183,Pentoshi 🐧 euroPeng 🇪🇺,Pentosh1,0,0,0.000000,0.000000,0.000000,800562,https://pbs.twimg.com/profile_images/176736682...,...,24.0,2.0,NaN,0.6850,0.1330,0.025900,NaN,1.0,3.0,16.0
183,183,Tom Schmidt ＞|＜,tomhschmidt,0,0,0.000000,0.000000,0.000000,34321,https://pbs.twimg.com/profile_images/177372148...,...,30.0,2.0,0.59,0.1500,0.0299,0.006995,1.0,1.0,6.0,6.0
184,183,natealex,natealexnft,0,0,0.000000,0.000000,0.000000,178429,https://pbs.twimg.com/profile_images/174301093...,...,10.0,2.0,NaN,0.1384,0.0300,0.006800,0.0,1.0,2.0,20.0
185,183,Kofi,0xKofi,0,0,0.000000,0.000000,0.000000,32354,https://pbs.twimg.com/profile_images/138786910...,...,28.0,2.0,NaN,0.1400,0.0269,0.006980,NaN,4.0,4.0,13.0


In [24]:
merged_df = merged_with_basic_hero_stats.merge(dataframes['last_trades'], on='hero_id', how='left')
final_merged_df = merged_df.merge(dataframes['tournament_scores'], left_on='hero_handle', right_on='Handle', how='left')
final_merged_df = final_merged_df.drop(columns=['Name', 'Handle'])

# Merge with hero_stats to include inflation_degree
final_merged_df = final_merged_df.merge(dataframes['hero_stats'][['hero_handle', 'inflation_degree']], 
                                        left_on='hero_handle', right_on='hero_handle', how='left')


final_merged_df['hero_id'] = final_merged_df['hero_id'].astype(str)
final_merged_df

,current_rank,hero_name,hero_handle,views,tweet_count,fantasy_score,reach,avg_views,hero_followers_count,hero_profile_image_url,...,All Rarities | 22 days,Main 4,Main 3,Common Only ✳️ Capped 15 🌟,Rare Only 💠,Common Only ✳️ Capped 20 🌟,Main 2,Main 1,Flash Tournament,inflation_degree
0,1,greg,greg16676935420,81523319,117,1000.000000,286.803798,696780.506403,1017227,https://pbs.twimg.com/profile_images/158101430...,...,955,949,862,900,941,"1,000","1,000",757,768,1.152022
1,2,GBH 🦁☝️ Remilio | ᚱᚹᛟ,xbtGBH,23941798,74,959.813585,467.486147,323537.812826,142979,https://pbs.twimg.com/profile_images/177972188...,...,986,930,992,"1,000",951,906,951,"1,000","1,000",1.152022
2,3,wallstreetbets,wallstreetbets,38745662,194,955.465391,101.985589,199719.909273,810707,https://pbs.twimg.com/profile_images/181575487...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,mitch (rtrd/acc),idrawline,5602370,125,939.614744,105.854892,44818.961453,65075,https://pbs.twimg.com/profile_images/177494222...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,based16z,based16z,3835380,267,916.626785,57.048130,14364.719070,35984,https://pbs.twimg.com/profile_images/179782076...,...,860,795,882,795,833,841,793,894,659,1.219587
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
182,183,Pentoshi 🐧 euroPeng 🇪🇺,Pentosh1,0,0,0.000000,0.000000,0.000000,800562,https://pbs.twimg.com/profile_images/176736682...,...,689,713,771,482,613,456,502,NaN,NaN,3.888415
183,183,Tom Schmidt ＞|＜,tomhschmidt,0,0,0.000000,0.000000,0.000000,34321,https://pbs.twimg.com/profile_images/177372148...,...,148,88,115,213,443,178,213,742,632,13.803617
184,183,natealex,natealexnft,0,0,0.000000,0.000000,0.000000,178429,https://pbs.twimg.com/profile_images/174301093...,...,90,139,131,59,146,54,67,116,178,12.469203
185,183,Kofi,0xKofi,0,0,0.000000,0.000000,0.000000,32354,https://pbs.twimg.com/profile_images/138786910...,...,363,657,225,26,442,723,599,NaN,NaN,1.050674


In [6]:
all_heroes_df = pd.read_csv('data/allHeroData.csv', dtype={'hero_id': str})

In [7]:
all_heroes_df['inflation_degree'].sum()

1773.4924074390035

In [9]:
all_heroes_stars_freq = all_heroes_df['hero_stars'].value_counts()
cumulative_sum_reverse = all_heroes_stars_freq.sort_index(ascending=False).cumsum()
cumulative_sum_reverse

hero_stars
7     27
6     57
5     86
4    115
3    143
2    177
Name: count, dtype: int64

## Calculating Best Decks with Scores

In [14]:
from itertools import combinations
import pandas as pd
from tqdm import tqdm
import pickle  # To save and load precomputed combinations

# Function to precompute all valid combinations and their scores
def precompute_combinations(df, max_stars, max_rarity_3_cards, score_columns):
    valid_combinations = []
    
    for combination in tqdm(combinations(df.index, 5), total=len(list(combinations(df.index, 5)))):
        combination_df = df.loc[list(combination)]
        
        total_stars = combination_df['hero_stars'].sum()
        rarity_3_count = combination_df['hero_rarity_index'].apply(lambda x: str(x).endswith('3')).sum()
        
        # Check if the combination meets the constraints
        hero_names = combination_df['hero_name']
        if (total_stars <= max_stars and rarity_3_count <= max_rarity_3_cards and
            len(set(hero_names)) == len(hero_names)):  # Ensure no duplicate heroes
            scores = {col: combination_df[col].sum() for col in score_columns}
            valid_combinations.append((combination, scores))
    
    return valid_combinations

# Function to find the best decks using precomputed combinations and card quantities
def find_best_decks(precomputed_combos, df, max_decks, score_columns):
    all_decks = {col: [] for col in score_columns}
    card_quantities = df['cards_number'].copy()

    for _ in range(max_decks):
        best_combination = {col: None for col in score_columns}
        best_scores = {col: 0 for col in score_columns}
        
        for combo_indices, scores in precomputed_combos:
            # Check if all cards in the combination are available in sufficient quantity
            if all(card_quantities[idx] > 0 for idx in combo_indices):
                for col in score_columns:
                    current_score = scores[col]
                    
                    # Update the best combination for each score column if the current one is better
                    if current_score > best_scores[col]:
                        best_combination[col] = combo_indices
                        best_scores[col] = current_score
        
        # If a valid combination is found, add it to the list of decks
        for col in score_columns:
            if best_combination[col] is not None:
                all_decks[col].append(df.loc[list(best_combination[col])])
                # Update the quantities of the used cards
                for idx in best_combination[col]:
                    card_quantities[idx] -= 1
            else:
                print(f"No more valid combinations found for {col}. Stopping.")
                return all_decks
    
    return all_decks

# Function to filter out already used cards
def filter_used_cards(combinations, used_cards):
    filtered_combos = []
    for combo_indices, scores in combinations:
        if not any(card in used_cards for card in combo_indices):
            filtered_combos.append((combo_indices, scores))
    return filtered_combos

# Load portfolio_scores
portfolio_scores = pd.read_csv('data/portfolio.csv')

# Parameters for Silver Tournament
max_stars = 22
max_rarity_3_cards = 3
max_decks = 4  # Number of decks you want to generate for Silver Tournament
score_columns = ['Main_Last_4_Ave']  # Only focusing on 'Main_Last_4_Ave'

# Precompute all valid combinations and their scores
precomputed_combos_silver = precompute_combinations(portfolio_scores, max_stars, max_rarity_3_cards, score_columns)

# Save precomputed combinations
with open('precomputed_combos_silver.pkl', 'wb') as f:
    pickle.dump(precomputed_combos_silver, f)

# Find the best decks for Silver Tournament
best_decks_silver = find_best_decks(precomputed_combos_silver, portfolio_scores, max_decks, score_columns)

# Display the results for each deck in Silver Tournament
for col in best_decks_silver:
    for i, deck in enumerate(best_decks_silver[col]):
        print(f"\nBest Combination DataFrame for {col}, Silver Deck {i+1}:")
        print(deck)
        print(f"Total {col} score: {deck[col].sum()}")

# Track used cards from Silver decks
used_cards_silver = set()
for deck in best_decks_silver['Main_Last_4_Ave']:
    used_cards_silver.update(deck.index)



100%|████████████████████████████████████████████████████████████████████| 5949147/5949147 [1:29:32<00:00, 1107.27it/s]



Best Combination DataFrame for Main_Last_4_Ave, Silver Deck 1:
          hero_name    hero_handle  hero_handle.1      hero_rarity_index  \
0           DCF GOD         dcfgod         dcfgod  1350996311777161219_3   
5   _gabrielShapir0       lex_node       lex_node   954015928924057601_3   
20  zac.eth 🧙🏻‍♂️♦️         zacxbt         zacxbt   886765719391789057_3   
33          Mercury  TraderMercury  TraderMercury  1180256836416614401_4   
42      Chumbawamba  chumbawamba22  chumbawamba22  1479247421330837508_4   

    cards_number  listed_cards_number  in_deck  gliding_score  card_in_deck  \
0              1                    0     True    1038.094843         False   
5              1                    0     True     845.291605          True   
20             1                    0     True     625.894136         False   
33             1                    0     True     478.133645         False   
42             1                    0     True     376.717765         False   

    

In [16]:
# Load precomputed combinations (instead of recomputing)
with open('precomputed_combos_silver.pkl', 'rb') as f:
    precomputed_combos_silver = pickle.load(f)

# Bronze Tournament: Filter precomputed combinations for those with only rarity 4 cards
def filter_bronze_combinations(precomputed_combos):
    filtered_combos = []
    
    for combo_indices, scores in tqdm(precomputed_combos, desc="Filtering Bronze combinations"):
        combo_rarities = portfolio_scores.loc[list(combo_indices), 'hero_rarity_index']
        
        # Only keep combinations where all cards are rarity 4
        if combo_rarities.apply(lambda x: str(x).endswith('4')).all():
            filtered_combos.append((combo_indices, scores))
    
    return filtered_combos

# Filter out combinations that use Silver cards for Bronze deck calculation
filtered_combos_bronze = filter_used_cards(precomputed_combos_silver, used_cards_silver)

# Apply the rarity 4 filter for Bronze decks
filtered_combos_bronze = filter_bronze_combinations(filtered_combos_bronze)

# Ensure Bronze deck does not exceed star limit
def filter_by_star_limit(combinations, max_stars):
    valid_combinations = []
    for combo_indices, scores in combinations:
        total_stars = portfolio_scores.loc[list(combo_indices), 'hero_stars'].sum()
        if total_stars <= max_stars:
            valid_combinations.append((combo_indices, scores))
    return valid_combinations

# Apply star limit for Bronze
filtered_combos_bronze = filter_by_star_limit(filtered_combos_bronze, 18)

# Find the best decks for Bronze Tournament
best_decks_bronze = find_best_decks(filtered_combos_bronze, portfolio_scores, 10, score_columns)

# Display the results for each deck in Bronze Tournament
for col in best_decks_bronze:
    for i, deck in enumerate(best_decks_bronze[col]):
        print(f"\nBest Combination DataFrame for {col}, Bronze Deck {i+1}:")
        print(deck)
        print(f"Total {col} score: {deck[col].sum()}")


Filtering Bronze combinations: 100%|█████████████████████████████████████████| 503762/503762 [05:12<00:00, 1614.22it/s]


No more valid combinations found for Main_Last_4_Ave. Stopping.

Best Combination DataFrame for Main_Last_4_Ave, Bronze Deck 1:
                    hero_name      hero_handle    hero_handle.1  \
9                     jingtao  muzzyvermillion  muzzyvermillion   
40            zac.eth 🧙🏻‍♂️♦️           zacxbt           zacxbt   
46                     Yano 🟪    JasonYanowitz    JasonYanowitz   
53  Unipcs (aka 'Bonk Guy') 🎒        theunipcs        theunipcs   
55                  KJ Crypto  koreanjewcrypto  koreanjewcrypto   

        hero_rarity_index  cards_number  listed_cards_number  in_deck  \
9   1769641180898131968_4             2                    1     True   
40   886765719391789057_4             2                    0     True   
46           1107518478_4             2                    0     True   
53  1755899659040555009_4             1                    0     True   
55   873190778910253056_4             2                    0     True   

    gliding_score  card_in_dec

In [13]:
best_decks_silver

{'Main_Last_4_Ave': [          hero_name    hero_handle  hero_handle.1      hero_rarity_index  \
  0           DCF GOD         dcfgod         dcfgod  1350996311777161219_3   
  5   _gabrielShapir0       lex_node       lex_node   954015928924057601_3   
  20  zac.eth 🧙🏻‍♂️♦️         zacxbt         zacxbt   886765719391789057_3   
  40  zac.eth 🧙🏻‍♂️♦️         zacxbt         zacxbt   886765719391789057_4   
  42      Chumbawamba  chumbawamba22  chumbawamba22  1479247421330837508_4   
  
      cards_number  listed_cards_number  in_deck  gliding_score  card_in_deck  \
  0              1                    0     True    1038.094843         False   
  5              1                    0     True     845.291605          True   
  20             1                    0     True     625.894136         False   
  40             2                    0     True     417.262757         False   
  42             1                    0     True     376.717765         False   
  
                     

In [3]:
import pickle  # To save and load precomputed combinations

# Save precomputed combinations
with open('precomputed_combos_silver.pkl', 'wb') as f:
    pickle.dump(precomputed_combos_silver, f)

# Load precomputed combinations (instead of recomputing)
with open('precomputed_combos_silver.pkl', 'rb') as f:
    precomputed_combos_silver = pickle.load(f)

In [5]:
# Function to filter out already used cards
def filter_used_cards(combinations, used_cards):
    filtered_combos = []
    for combo_indices, scores in combinations:
        if not any(card in used_cards for card in combo_indices):
            filtered_combos.append((combo_indices, scores))
    return filtered_combos

# Load portfolio_scores
portfolio_scores = pd.read_csv('data/portfolio.csv')

# Track used cards from Silver decks
used_cards_silver = set()
for deck in best_decks_silver['Main_Last_4_Ave']:
    used_cards_silver.update(deck.index)

# Filter out combinations that use Silver cards for Bronze deck calculation
filtered_combos_bronze = filter_used_cards(precomputed_combos_silver, used_cards_silver)

# Ensure Bronze deck does not exceed star limit
def filter_by_star_limit(combinations, max_stars):
    valid_combinations = []
    for combo_indices, scores in combinations:
        total_stars = portfolio_scores.loc[list(combo_indices), 'hero_stars'].sum()
        if total_stars <= max_stars:
            valid_combinations.append((combo_indices, scores))
    return valid_combinations

# Apply star limit for Bronze
filtered_combos_bronze = filter_by_star_limit(filtered_combos_bronze, max_stars_bronze)

# Now proceed to generate Bronze decks using the filtered combinations
best_decks_bronze = find_best_decks(filtered_combos_bronze, portfolio_scores, max_decks_bronze, score_columns)

# Display the results for each deck in Bronze Tournament
for col in best_decks_bronze:
    for i, deck in enumerate(best_decks_bronze[col]):
        print(f"\nBest Combination DataFrame for {col}, Bronze Deck {i+1}:")
        print(deck)
        print(f"Total {col} score: {deck[col].sum()}")


Best Combination DataFrame for Main_Last_4_Ave, Bronze Deck 1:
               hero_name     hero_handle      hero_rarity_index  cards_number  \
4   Wizard Of SoHo (🍷,🍷)    wizardofsoho  1175781003245191178_4             1   
32                  Adam  surfcoderepeat            408488225_3             1   
33       kipit | fan/acc         0xKipit   720313843021385728_3             1   
62                 Herro     HerroCrypto   949016786506100736_4             4   
63               foobar/        0xfoobar  1356386110847377408_3             2   

    listed_cards_number  in_deck  gliding_score  card_in_deck  \
4                     0    False     878.345553         False   
32                    0     True     473.938113         False   
33                    0     True     460.175090         False   
62                    0    False     163.535235         False   
63                    0    False     146.043560         False   

                                          picture_url  rar

### Work Out Best Bronze

In [ ]:

print("Step 1: Filter rows where hero_rarity_index ends in '4'")

# Filter rows where hero_rarity_index ends in '4'
filtered_df = portfolio_scores[portfolio_scores['hero_rarity_index'].astype(str).str.endswith('4')]

print(f"Filtered DataFrame has {len(filtered_df)} rows")

print("Step 2: Generate all combinations of 5 cards")
# Generate all combinations of 5 cards
combinations = list(itertools.combinations(filtered_df.index, 5))

print(f"Generated {len(combinations)} combinations")

# Define the columns for which we want to find the optimum combination
score_columns = ['hero_fantasy_score', 'Main 8', 'Main 7', 'Average', 'Main_Tournaments_Ave']

# Initialize variables to store the best combination and its scores
best_combination = {col: None for col in score_columns}
best_scores = {col: float('-inf') for col in score_columns}

print("Step 3: Evaluate each combination")
# Evaluate each combination with a progress bar
for combo in tqdm(combinations, desc="Evaluating combinations"):
    combo_df = filtered_df.loc[list(combo)]
    total_stars = combo_df['hero_stars'].sum()
    
    # Check if the combination meets the hero_stars constraint
    if total_stars <= 18:
        for col in score_columns:
            score = combo_df[col].sum()
            
            if score > best_scores[col]:
                best_scores[col] = score
                best_combination[col] = combo_df

print("Step 4: Print the best combinations and their scores")
# Print the best combinations and their scores
for col in score_columns:
    print(f"Best combination for {col}:")
    print(best_combination[col])
    print(f"Total {col} score: {best_scores[col]}\n")

Step 1: Filter rows where hero_rarity_index ends in '4'


NameError: name 'portfolio_scores' is not defined

In [ ]:

best_combination['Average'].to_csv('bronze_best_Average.csv', index=False)

### Work out best silver from own cards

In [ ]:
import pandas as pd
from itertools import combinations
from tqdm import tqdm

# Function to find the best combination of cards
def find_best_combinations(df, max_stars, max_rarity_3_cards):
    score_columns = ['Average', 'Main_Tournaments_Ave']
    
    # Debug: Print the columns of the DataFrame
    print("Columns in DataFrame:", df.columns.tolist())
    
    for col in score_columns:
        if col not in df.columns:
            raise KeyError(f"Column '{col}' not found in DataFrame")
    
    best_combinations = {col: None for col in score_columns}
    best_scores = {col: 0 for col in score_columns}
    
    # Use tqdm to track progress
    print("Starting combination search...")
    for combination in tqdm(combinations(df.index, 5), total=len(list(combinations(df.index, 5)))):
        combination_df = df.loc[list(combination)]
        
        total_stars = combination_df['hero_stars'].sum()
        rarity_3_count = combination_df['hero_rarity_index'].apply(lambda x: str(x).endswith('3')).sum()
        
        # Check if the combination meets the constraints
        if total_stars <= max_stars and rarity_3_count <= max_rarity_3_cards:
            for col in score_columns:
                current_score = combination_df[col].sum()
                
                # Update the best combination for each score column if the current one is better
                if current_score > best_scores[col]:
                    best_combinations[col] = combination_df
                    best_scores[col] = current_score

    return best_combinations, best_scores

# Parameters
max_stars = 22
max_rarity_3_cards = 3

# Finding the best combinations
best_combinations, best_scores = find_best_combinations(portfolio_scores, max_stars, max_rarity_3_cards)

for col in best_combinations:
    print(f"\nBest Combination DataFrame for {col}:")
    print(best_combinations[col])
    print(f"Best Score for {col}: {best_scores[col]}")


Columns in DataFrame: ['owner', 'hero_rarity_index', 'cards_number', 'listed_cards_number', 'in_deck', 'card_id', 'card_owner', 'gliding_score', 'card_in_deck', 'picture_url', 'token_id', 'rarity', 'hero_id', 'hero_name', 'hero_handle', 'hero_profile_image_url', 'hero_followers_count', 'hero_stars', 'hero_fantasy_score', 'hero_views', 'hero_current_rank', 'hero_trades', 'Main 8', 'Main 7', 'Main 6', 'Main 5', 'All Rarities 22 days', 'Main 4', 'Main 3', 'Common Only Capped 15', 'Rare Only', 'Common Only ?? Capped 20 ??', 'Main 2', 'Main 1', 'Flash Tournament', 'Average', 'Main_Tournaments_Ave']
Starting combination search...


100%|█████████████████████████████████████████████████████████████████████| 6471002/6471002 [2:40:01<00:00, 673.97it/s]


Best Combination DataFrame for Average:
                                         owner      hero_rarity_index  \
4   0xDC0c171F4DB2790e565295c5287bCa9D4071EA1a            588569122_4   
8   0xDC0c171F4DB2790e565295c5287bCa9D4071EA1a   886765719391789057_3   
9   0xDC0c171F4DB2790e565295c5287bCa9D4071EA1a   954015928924057601_3   
38  0xDC0c171F4DB2790e565295c5287bCa9D4071EA1a           2593497397_4   
49  0xDC0c171F4DB2790e565295c5287bCa9D4071EA1a  1356386110847377408_3   

    cards_number  listed_cards_number  in_deck  \
4              1                    0     True   
8              1                    0     True   
9              1                    0     True   
38             1                    0     True   
49             2                    0     True   

                                              card_id  \
4   475494_0x0AAADCf421A3143E5cB2dDB8452c03ae595B0734   
8   280224_0x0AAADCf421A3143E5cB2dDB8452c03ae595B0734   
9   408340_0x0AAADCf421A3143E5cB2dDB8452c03ae595

In [ ]:
best_combinations['Main_Tournaments_Ave'].to_csv('silver_best_Main_Tournaments_Ave.csv', index=False)

## Merge Upcoming Entries with Scores

In [ ]:
# Combine the Bronze and Silver DataFrames
upcoming_Bronze = dataframes['upcoming_Bronze']
upcoming_Silver = dataframes['upcoming_Silver']

# Add a column to indicate the source
upcoming_Bronze['tournament_type'] = 'Bronze'
upcoming_Silver['tournament_type'] = 'Silver'

# Combine the DataFrames
combined_df = pd.concat([upcoming_Bronze, upcoming_Silver], ignore_index=True)

# Merge the combined DataFrame with tournament_scores DataFrame
tournament_scores_df = dataframes['tournament_scores']
merged_df = combined_df.merge(tournament_scores_df, left_on='hero_handle', right_on='Handle', how='left')

# Drop the 'Name' and 'Handle' columns from the merged DataFrame
merged_df = merged_df.drop(['Name', 'Handle'], axis=1)

merged_df.columns


Index(['entry_id', 'card_id', 'card_owner', 'gliding_score', 'card_in_deck',
       'picture_url', 'token_id', 'hero_rarity_index', 'hero_id', 'hero_name',
       'hero_handle', 'hero_profile_image_url', 'hero_followers_count',
       'hero_stars', 'hero_fantasy_score', 'hero_current_rank',
       'tournament_fantasy_score', 'tournament_current_rank',
       'tournament_views', 'tournament_type', 'Main 8', 'Main 7', 'Main 6',
       'Main 5', 'All Rarities 22 days', 'Main 4', 'Main 3',
       'Common Only Capped 15', 'Rare Only', 'Common Only ?? Capped 20 ??',
       'Main 2', 'Main 1', 'Flash Tournament', 'Average',
       'Main_Tournaments_Ave'],
      dtype='object')

In [ ]:

# Add the entry_name column
merged_df['entry_name'] = merged_df['tournament_type'] + ' ' + merged_df['entry_id'].astype(str)

# List of columns to be adjusted
score_columns = ['hero_fantasy_score', 'Main 8', 'Main 7', 'Main 6', 'Main 5', 
                 'All Rarities 22 days', 'Main 4', 'Main 3', 'Common Only Capped 15', 
                 'Rare Only', 'Common Only ?? Capped 20 ??', 'Main 2', 'Main 1', 
                 'Flash Tournament', 'Average', 'Main_Tournaments_Ave']

# Convert all score columns to numeric, coercing errors to NaNs
for col in score_columns:
    merged_df[col] = pd.to_numeric(merged_df[col], errors='coerce')

# Apply the multiplier before grouping
for col in score_columns:
    merged_df[col] = merged_df.apply(lambda row: row[col] * 1.5 if row['hero_rarity_index'].endswith('3') and row['tournament_type'] == 'Silver' else row[col], axis=1)

# Function to sum columns while respecting NaNs
def sum_with_nan_check(group):
    result = {}
    for col in score_columns:
        if group[col].isnull().any():
            result[col] = float('nan')
        else:
            result[col] = group[col].sum()
    return pd.Series(result)

# Group by 'entry_name' and apply the sum_with_nan_check function
grouped_df = merged_df.groupby('entry_name').apply(sum_with_nan_check).reset_index()


C:\Users\beuzi\AppData\Local\Temp\ipykernel_17276\2514957361.py:29: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  grouped_df = merged_df.groupby('entry_name').apply(sum_with_nan_check).reset_index()


In [ ]:
grouped_df

,entry_name,hero_fantasy_score,Main 8,Main 7,Main 6,Main 5,All Rarities 22 days,Main 4,Main 3,Common Only Capped 15,Rare Only,Common Only ?? Capped 20 ??,Main 2,Main 1,Flash Tournament,Average,Main_Tournaments_Ave
0,Bronze 1,2194.320617,2585.0,2505.0,3768.0,3132.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2705.125399,2684.416667
1,Bronze 10,1979.917087,2184.0,2343.0,2679.0,2411.0,2488.0,2565.0,2712.0,2478.0,2544.0,2703.0,2471.0,NaN,NaN,2449.200181,2375.750000
2,Bronze 2,2078.571725,1867.0,2745.0,2371.0,2411.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2323.932862,2340.166667
3,Bronze 3,1911.823184,1875.0,2242.0,1775.0,2088.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2077.836674,2123.833333
4,Bronze 4,1648.232070,1820.0,2241.0,2780.0,2526.0,2191.0,2529.0,1988.0,2696.0,2020.0,2425.0,2340.0,NaN,NaN,2255.877893,2279.000000
5,Bronze 5,1923.814305,1967.0,1988.0,1688.0,1698.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2049.024850,1900.250000
6,Bronze 6,2296.763519,2057.0,1844.0,2348.0,2662.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2739.781806,2378.333333
7,Bronze 7,1831.271398,2081.0,2060.0,2785.0,2665.0,2619.0,2586.0,2744.0,3080.0,2175.0,2022.0,1701.0,NaN,NaN,2387.252156,2348.000000
8,Bronze 8,2176.451479,2574.0,1700.0,2278.0,1677.0,2393.0,2086.0,2271.0,2489.0,2372.0,2713.0,2524.0,NaN,NaN,2315.866229,2009.250000
9,Bronze 9,1676.719610,1405.0,1596.0,2245.0,2682.0,2237.0,2298.0,2600.0,2332.0,2456.0,2057.0,1956.0,2665.0,2210.0,2195.371966,1995.250000


In [ ]:
grouped_df[grouped_df[

In [ ]:
# # Filter the DataFrame for rows where tournament_type is 'Silver'
# filtered_df = merged_df[merged_df['tournament_type'] == 'Silver']

# # Select only the 'hero_id' and 'hero_fantasy_score' columns
# result_df = filtered_df[['hero_name', 'hero_fantasy_score']]

# # Display the resulting DataFrame
# print(result_df)

                hero_name  hero_fantasy_score
50                PreRich         1010.202513
51        _gabrielShapir0          837.883175
52  Pacman | Blur + Blast          573.357884
53              K A L E O          731.187600
54              KJ Crypto          387.578233
55                   Iced          795.019341
56        Hugo Martingale          788.938727
57                foobar/          316.421788
58               ciniz 👁️          599.511482
59                 Yano 🟪          665.516454
60           yieldfarming          903.579790
61        zac.eth 🧙🏻‍♂️♦️          781.841804
62                  apix🎮          669.451981
63                   Minh          372.268968
64        _gabrielShapir0          558.588783
65                 kmoney          898.002957
66                     qw          625.521905
67                foobar/          316.421788
68                   Iced          530.012894
69               AiRev //          444.548694
70           yieldfarming         

In [ ]:
dataframes['upcoming_Silver']

,entry_id,card_id,card_owner,gliding_score,card_in_deck,picture_url,token_id,hero_rarity_index,hero_id,hero_name,hero_handle,hero_profile_image_url,hero_followers_count,hero_stars,hero_fantasy_score,hero_current_rank,tournament_fantasy_score,tournament_current_rank,tournament_views,tournament_type
0,1,254484_0x0AAADCf421A3143E5cB2dDB8452c03ae595B0734,0xDC0c171F4DB2790e565295c5287bCa9D4071EA1a,1010.202513,True,https://fantasy-top-cards.s3.eu-north-1.amazon...,254484,1585936726370852865_3,1585936726370852865,PreRich,0xprerich,https://pbs.twimg.com/profile_images/181170797...,15568,5,673.468342,39,0,0,0,Silver
1,1,429727_0x0AAADCf421A3143E5cB2dDB8452c03ae595B0734,0xDC0c171F4DB2790e565295c5287bCa9D4071EA1a,837.883175,True,https://fantasy-top-cards.s3.eu-north-1.amazon...,429727,954015928924057601_3,954015928924057601,_gabrielShapir0,lex_node,https://pbs.twimg.com/profile_images/176506460...,47194,5,558.588783,70,0,0,0,Silver
2,1,144702_0x0AAADCf421A3143E5cB2dDB8452c03ae595B0734,0xDC0c171F4DB2790e565295c5287bCa9D4071EA1a,573.357884,True,https://fantasy-top-cards.s3.eu-north-1.amazon...,144702,1492434155405287424_3,1492434155405287424,Pacman | Blur + Blast,PacmanBlur,https://pbs.twimg.com/profile_images/149957170...,60294,3,382.238589,113,0,0,0,Silver
3,1,495229_0x0AAADCf421A3143E5cB2dDB8452c03ae595B0734,0xDC0c171F4DB2790e565295c5287bCa9D4071EA1a,731.187600,True,https://fantasy-top-cards.s3.eu-north-1.amazon...,495229,906234475604037637_4,906234475604037637,K A L E O,CryptoKaleo,https://pbs.twimg.com/profile_images/160604173...,642260,6,731.187600,28,0,0,0,Silver
4,1,272792_0x0AAADCf421A3143E5cB2dDB8452c03ae595B0734,0xDC0c171F4DB2790e565295c5287bCa9D4071EA1a,387.578233,True,https://fantasy-top-cards.s3.eu-north-1.amazon...,272792,873190778910253056_4,873190778910253056,KJ Crypto,koreanjewcrypto,https://pbs.twimg.com/profile_images/181511037...,137404,3,387.578233,110,0,0,0,Silver
5,2,197263_0x0AAADCf421A3143E5cB2dDB8452c03ae595B0734,0xDC0c171F4DB2790e565295c5287bCa9D4071EA1a,795.019341,True,https://fantasy-top-cards.s3.eu-north-1.amazon...,197263,618539620_3,618539620,Iced,IcedKnife,https://pbs.twimg.com/profile_images/165074681...,311853,6,530.012894,79,0,0,0,Silver
6,2,262872_0x0AAADCf421A3143E5cB2dDB8452c03ae595B0734,0xDC0c171F4DB2790e565295c5287bCa9D4071EA1a,788.938727,True,https://fantasy-top-cards.s3.eu-north-1.amazon...,262872,1635676907529285642_3,1635676907529285642,Hugo Martingale,HugoMartingale,https://pbs.twimg.com/profile_images/179487982...,14263,4,525.959152,81,0,0,0,Silver
7,2,434003_0x0AAADCf421A3143E5cB2dDB8452c03ae595B0734,0xDC0c171F4DB2790e565295c5287bCa9D4071EA1a,316.421788,True,https://fantasy-top-cards.s3.eu-north-1.amazon...,434003,1356386110847377408_3,1356386110847377408,foobar/,0xfoobar,https://pbs.twimg.com/profile_images/174582432...,155394,2,210.947858,155,0,0,0,Silver
8,2,191364_0x0AAADCf421A3143E5cB2dDB8452c03ae595B0734,0xDC0c171F4DB2790e565295c5287bCa9D4071EA1a,599.511482,True,https://fantasy-top-cards.s3.eu-north-1.amazon...,191364,1093242943127932930_4,1093242943127932930,ciniz 👁️,screentimes,https://pbs.twimg.com/profile_images/180397688...,74095,5,599.511482,58,0,0,0,Silver
9,2,436924_0x0AAADCf421A3143E5cB2dDB8452c03ae595B0734,0xDC0c171F4DB2790e565295c5287bCa9D4071EA1a,665.516454,True,https://fantasy-top-cards.s3.eu-north-1.amazon...,436924,1107518478_4,1107518478,Yano 🟪,JasonYanowitz,https://pbs.twimg.com/profile_images/176861934...,111219,4,665.516454,42,0,0,0,Silver


## Portfolio Bits

In [ ]:

# Assuming 'dataframes' is a dictionary with your DataFrames and 'final_merged_df' is your DataFrame
portfolio_df = dataframes['portfolio']
final_df = final_merged_df

# Step 1: Extract the last digit from 'hero_rarity_index' and create 'rarity' column
portfolio_df['rarity'] = portfolio_df['hero_rarity_index'].str[-1]

# Step 2: Remove 'hero_rarity_index' column
portfolio_df = portfolio_df.drop(columns=['hero_rarity_index'])

# Step 3: Add 'lowest price' column
def get_lowest_price(row, final_df):
    rarity_column = f'rarity{row["rarity"]}_lowest_price'
    matched_row = final_df[final_df['hero_id'] == row['hero_id']]
    if not matched_row.empty:
        return matched_row.iloc[0][rarity_column]
    return None

portfolio_df['lowest price'] = portfolio_df.apply(get_lowest_price, final_df=final_df, axis=1)

# Step 4: Add 'last price' column
def get_last_price(row, final_df):
    rarity_column = f'rarity{row["rarity"]}lastSalePrice'
    matched_row = final_df[final_df['hero_id'] == row['hero_id']]
    if not matched_row.empty:
        return matched_row.iloc[0][rarity_column]
    return None

portfolio_df['last price'] = portfolio_df.apply(get_last_price, final_df=final_df, axis=1)

# Update the dictionary with the modified DataFrame
dataframes['portfolio'] = portfolio_df

# Display the updated DataFrame
print(portfolio_df)

KeyError: 'rarity3_lowest_price'

In [ ]:
# Step 1: Create new columns for the products
portfolio_df['total_value_lowest_price'] = portfolio_df['lowest price'] * portfolio_df['cards_number']
portfolio_df['total_value_last_price'] = portfolio_df['last price'] * portfolio_df['cards_number']

# Step 2: Calculate the sums
total_lowest_price_value = portfolio_df['total_value_lowest_price'].sum()
total_last_price_value = portfolio_df['total_value_last_price'].sum()

# Display the results
print(f"Total Lowest Price Value: {total_lowest_price_value}")
print(f"Total Last Price Value: {total_last_price_value}")

Total Lowest Price Value: 2.6838221291111006
Total Last Price Value: 2.3893410904999994
